In [28]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
import mat_ceng as mc
import numpy as np
import pandas as pd
import math

### table names
+ Frame Assigns - Sect Prop
+ Frame Assigns - Summary
+ Frame Sec Def - Conc Col Reinf
+ Frame Sec Def - Conc Rect
+ Mat Prop - Basic Mech Props
+ Mat Prop - Concrete Data
+ Mat Prop - Rebar Data
---
+ Design Forces - Columns

In [43]:
excel_file = 'column_design.xlsx'
df = pd.read_excel(io=excel_file,sheet_name='Frame Assigns - Sect Prop',
                   skiprows=[0, 2], usecols=[2,3,5])
display(df)
df = pd.read_excel(io=excel_file,sheet_name='Frame Assigns - Summary',
                   skiprows=[0,2],usecols=[2,4,5,7]).fillna(value=0)
display(df)
df = pd.read_excel(io=excel_file,sheet_name='Frame Sec Def - Conc Col Reinf',
                   skiprows=[0, 2],usecols=[0,3,6,7,8,9,10,11,12])
display(df)
df = pd.read_excel(io=excel_file,sheet_name='Frame Sec Def - Conc Rect',
                   skiprows=[0, 2], usecols=[0,1,3,4,8])
display(df)
df = pd.read_excel(io=excel_file,sheet_name='Mat Prop - Concrete Data',
                   skiprows=[0, 2], usecols=[0,1])
df

,UniqueName,Shape,Section Property
0,1234,Concrete Rectangular,B45*60
1,1231,Concrete Rectangular,B45*60
2,1232,Concrete Rectangular,B45*60
3,1235,Concrete Rectangular,B45*60
4,1233,Concrete Rectangular,B45*60
...,...,...,...
5897,5166,Concrete Rectangular,C600*1000
5898,5165,Concrete Rectangular,C600*1000
5899,5163,Concrete Rectangular,C600*1000
5900,256,Concrete Rectangular,C1200*1200


,UniqueName,Length,Analysis Section,Axis Angle
0,1234,1844.20,B45*60,0.000
1,1231,5219.91,B45*60,0.000
2,1232,2241.35,B45*60,0.000
3,1235,6766.89,B45*60,0.000
4,1233,7170.36,B45*60,0.000
...,...,...,...,...
5897,5166,6250.00,C600*1000,-97.414
5898,5165,6250.00,C600*1000,-97.414
5899,5163,6250.00,C600*1000,412.653
5900,256,6250.00,C1200*1200,322.296


,Name,Reinforcement Configuration,Clear Cover to Ties,Number Bars 3-Dir,Number Bars 2-Dir,Number Longitudinal Bars,Longitudinal Bar Size,Corner Bar Size,Tie Bar Size
0,C 1500*600,Rectangular,40,12.0,5.0,NaN,25,25.0,10
1,C 650*650,Rectangular,40,5.0,5.0,NaN,25,25.0,10
2,C1000,Circular,40,NaN,NaN,36.0,25,NaN,10
3,C1000 SPIRAL,Circular,40,NaN,NaN,36.0,25,NaN,10
4,C1000*1000,Rectangular,40,9.0,9.0,NaN,25,25.0,10
...,...,...,...,...,...,...,...,...,...
168,ConcBm2,Rectangular,40,3.0,3.0,NaN,20,20.0,10
169,ConcBm3,Rectangular,40,3.0,3.0,NaN,20,20.0,10
170,ConcBm4,Rectangular,40,3.0,3.0,NaN,20,20.0,10
171,ConcBm5,Rectangular,40,3.0,3.0,NaN,20,20.0,10


,Name,Material,Depth,Width,Design Type
0,B 40*62,C40,620.0,400.0,Beam
1,B 40*68,C40,680.0,400.0,Beam
2,B 45*132,C40,1320.0,450.0,Beam
3,B 45*180,C40,1800.0,450.0,Beam
4,B 45*186,C40,1860.0,450.0,Beam
...,...,...,...,...,...
452,FSec5,OTHER,449.8,1212.0,Column
453,FSec6,OTHER,600.0,1400.0,Column
454,FSec7,OTHER,450.0,800.0,Column
455,FSec8,OTHER,801.1,450.0,Column


,Material,Fc
0,C40,40
1,C50,50
2,C60,60


In [44]:
df_frames:pd.DataFrame
df_sections:pd.DataFrame

df_frames = pd.read_excel(io=excel_file,sheet_name='Frame Assigns - Sect Prop',
                   skiprows=[0, 2], usecols=[2,3,5])
df_frames = df_frames[df_frames['Shape'] == 'Concrete Rectangular']

df = pd.read_excel(io=excel_file,sheet_name='Frame Assigns - Summary',
                   skiprows=[0,2],usecols=[2,4])
df_frames = df_frames.merge(df,how='inner', on='UniqueName').drop(axis=1, labels=['Shape'])
display(df_frames)
df_sections = pd.read_excel(io=excel_file,sheet_name='Frame Sec Def - Conc Col Reinf',
                   skiprows=[0, 2],usecols=[0,3,6,7,8,10,12])
df_sections = df_sections [df_sections['Reinforcement Configuration'] == 'Rectangular']

df = pd.read_excel(io=excel_file,sheet_name='Frame Sec Def - Conc Rect',
                   skiprows=[0, 2], usecols=[0,1,3,4,8])
df = df[df['Design Type'] == 'Column']
df_sections = df_sections.merge(df,how='inner', on='Name')

df = pd.read_excel(io=excel_file,sheet_name='Mat Prop - Concrete Data',
                   skiprows=[0, 2], usecols=[0,1])
df_sections = df_sections.merge(df,how='inner', on='Material').drop(axis=1, labels=['Reinforcement Configuration','Material','Design Type'])
df_sections['As'] = (2 * (df_sections['Number Bars 3-Dir'] + df_sections['Number Bars 2-Dir']) - 4) * math.pi / 4 * df_sections['Longitudinal Bar Size'] **2
df_sections['rft_ratio'] = df_sections['As'] / (df_sections['Depth'] * df_sections['Width'])
display(df_sections)
df_input = df_frames.merge(df_sections,how='inner', left_on='Section Property', right_on='Name').drop(axis=1, labels=['Name'])
df_input

,UniqueName,Section Property,Length
0,1234,B45*60,1844.20
1,1231,B45*60,5219.91
2,1232,B45*60,2241.35
3,1235,B45*60,6766.89
4,1233,B45*60,7170.36
...,...,...,...
5819,5166,C600*1000,6250.00
5820,5165,C600*1000,6250.00
5821,5163,C600*1000,6250.00
5822,256,C1200*1200,6250.00


,Name,Clear Cover to Ties,Number Bars 3-Dir,Number Bars 2-Dir,Longitudinal Bar Size,Tie Bar Size,Depth,Width,Fc,As,rft_ratio
0,C 1500*600,40,12.0,5.0,25,10,600.0,1500.0,50,14726.215564,0.016362
1,C 650*650,40,5.0,5.0,25,10,650.0,650.0,50,7853.981634,0.018589
2,C1000*1000,40,9.0,9.0,25,10,1000.0,1000.0,50,15707.963268,0.015708
3,C1000*1000 (36T25),40,10.0,10.0,25,10,1000.0,1000.0,50,17671.458676,0.017671
4,C1000*1200,40,9.0,11.0,25,10,1200.0,1000.0,50,17671.458676,0.014726
...,...,...,...,...,...,...,...,...,...,...,...
155,ConcBm1,40,3.0,3.0,20,10,450.0,450.0,40,2513.274123,0.012411
156,ConcBm2,40,3.0,3.0,20,10,450.0,450.0,40,2513.274123,0.012411
157,ConcBm3,40,3.0,3.0,20,10,450.0,450.0,40,2513.274123,0.012411
158,ConcBm4,40,3.0,3.0,20,10,450.0,450.0,40,2513.274123,0.012411


,UniqueName,Section Property,Length,Clear Cover to Ties,Number Bars 3-Dir,Number Bars 2-Dir,Longitudinal Bar Size,Tie Bar Size,Depth,Width,Fc,As,rft_ratio
0,4533,C450*600 M,2450.0,40,4.0,5.0,20,10,600.0,450.0,50,4398.229715,0.016290
1,4531,C450*1200 (30T25) M,2450.0,40,5.0,12.0,25,10,1200.0,450.0,50,14726.215564,0.027271
2,4534,C450*1000,2450.0,40,4.0,9.0,25,10,1000.0,450.0,50,10799.224747,0.023998
3,4535,C450*1000 M,2450.0,40,4.0,9.0,25,10,1000.0,450.0,50,10799.224747,0.023998
4,4532,C450*1000 M,2450.0,40,4.0,9.0,25,10,1000.0,450.0,50,10799.224747,0.023998
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1358,5166,C600*1000,6250.0,40,5.0,7.0,25,10,1000.0,600.0,50,9817.477042,0.016362
1359,5165,C600*1000,6250.0,40,5.0,7.0,25,10,1000.0,600.0,50,9817.477042,0.016362
1360,5163,C600*1000,6250.0,40,5.0,7.0,25,10,1000.0,600.0,50,9817.477042,0.016362
1361,256,C1200*1200,6250.0,40,12.0,12.0,25,10,1200.0,1200.0,50,21598.449493,0.014999


In [45]:
df_forces = pd.read_excel(io=excel_file,sheet_name='Design Forces - Columns',
                   skiprows=[0, 2], usecols=[2,3,4,5,6,7,8,9,10])
df_forces

,UniqueName,Combo,Station,P,V2,V3,T,M2,M3
0,4533,1.4DL+1.7LL-1,0.00,-385694.64,37579.86,6258.48,-22441645.66,15169347.66,-4.431966e+07
1,4533,1.4DL+1.7LL-1,925.02,-376953.18,37579.86,6258.48,-22441645.66,9439164.92,-7.901675e+07
2,4533,1.4DL+1.7LL-1,1850.04,-368211.72,37579.86,6258.48,-22441645.66,3661113.19,-1.134883e+08
3,4533,1.4DL+1.7LL-2,0.00,-385694.64,37579.86,6258.48,-22441645.66,15169347.66,-4.431966e+07
4,4533,1.4DL+1.7LL-2,925.02,-376953.18,37579.86,6258.48,-22441645.66,9439164.92,-7.901675e+07
...,...,...,...,...,...,...,...,...,...
542075,5880,0.9D-1,1625.00,-1323063.28,33131.65,-4980.64,-1887.38,-1784357.85,1.285250e+07
542076,5880,0.9D-1,3250.00,-1293813.28,33131.65,-4980.64,-1887.38,6339378.67,-4.112411e+07
542077,5880,1.4D-1,0.00,-2103598.44,51538.12,-7747.66,-2935.93,-15397928.41,1.038888e+08
542078,5880,1.4D-1,1625.00,-2058098.44,51538.12,-7747.66,-2935.93,-2775667.76,1.999278e+07


In [48]:
df_final = df_forces.merge(df_input, how='inner', on='UniqueName')
df_final

,UniqueName,Combo,Station,P,V2,V3,T,M2,M3,Section Property,...,Clear Cover to Ties,Number Bars 3-Dir,Number Bars 2-Dir,Longitudinal Bar Size,Tie Bar Size,Depth,Width,Fc,As,rft_ratio
0,4533,1.4DL+1.7LL-1,0.00,-385694.64,37579.86,6258.48,-22441645.66,15169347.66,-4.431966e+07,C450*600 M,...,40,4.0,5.0,20,10,600.0,450.0,50,4398.229715,0.016290
1,4533,1.4DL+1.7LL-1,925.02,-376953.18,37579.86,6258.48,-22441645.66,9439164.92,-7.901675e+07,C450*600 M,...,40,4.0,5.0,20,10,600.0,450.0,50,4398.229715,0.016290
2,4533,1.4DL+1.7LL-1,1850.04,-368211.72,37579.86,6258.48,-22441645.66,3661113.19,-1.134883e+08,C450*600 M,...,40,4.0,5.0,20,10,600.0,450.0,50,4398.229715,0.016290
3,4533,1.4DL+1.7LL-2,0.00,-385694.64,37579.86,6258.48,-22441645.66,15169347.66,-4.431966e+07,C450*600 M,...,40,4.0,5.0,20,10,600.0,450.0,50,4398.229715,0.016290
4,4533,1.4DL+1.7LL-2,925.02,-376953.18,37579.86,6258.48,-22441645.66,9439164.92,-7.901675e+07,C450*600 M,...,40,4.0,5.0,20,10,600.0,450.0,50,4398.229715,0.016290
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
511275,5880,0.9D-1,1625.00,-1323063.28,33131.65,-4980.64,-1887.38,-1784357.85,1.285250e+07,C800*1000,...,40,7.0,9.0,25,10,1000.0,800.0,50,13744.467859,0.017181
511276,5880,0.9D-1,3250.00,-1293813.28,33131.65,-4980.64,-1887.38,6339378.67,-4.112411e+07,C800*1000,...,40,7.0,9.0,25,10,1000.0,800.0,50,13744.467859,0.017181
511277,5880,1.4D-1,0.00,-2103598.44,51538.12,-7747.66,-2935.93,-15397928.41,1.038888e+08,C800*1000,...,40,7.0,9.0,25,10,1000.0,800.0,50,13744.467859,0.017181
511278,5880,1.4D-1,1625.00,-2058098.44,51538.12,-7747.66,-2935.93,-2775667.76,1.999278e+07,C800*1000,...,40,7.0,9.0,25,10,1000.0,800.0,50,13744.467859,0.017181


In [ ]:
# sample
'''

mini_df['E'] = mini_df.apply(
    lambda s: concrete_elastic_modulus(s['fc_MPa'], s['gamma']),
    axis=1
)

'''
